<a href="https://colab.research.google.com/github/Paganini134/Deep-Learning1/blob/main/MovieRecommender_pynb_new.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import ast
import csv
import sklearn as skt
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import RandomOverSampler


In [ ]:
movies=pd.read_csv("tmdb_5000_movies.csv",error_bad_lines=False, engine="python")
#credits= pd.read_csv("tmdb_5000_credits.csv")
print(len(movies))

4803


<ipython-input-173-1ce93e811a37>:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  movies=pd.read_csv("tmdb_5000_movies.csv",error_bad_lines=False, engine="python")


In [ ]:
!pip install nltk

In [ ]:
new="newdata.csv"
cols=["movie_id","title","cast","crew"]
credits=pd.read_csv(new, error_bad_lines=False, engine="python")
credits.head()

print(len(credits))

<ipython-input-175-e5abd33bfc0a>:3: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  credits=pd.read_csv(new, error_bad_lines=False, engine="python")


4803


In [ ]:
#Removing duplicate data and empty data
print(len(movies))
#movies.isnull().sum()

#sum over all columns
#movies.dropna(inplace=True)
print(len(movies))

4803
4803


In [ ]:
movies=movies.merge(credits,on="title")

In [ ]:
#cast_dict=movies['cast']
#considering a row at a time
movies.iloc[0].genres
#you can convert a dataframe to a python dictionary
#REVISITTTT

'[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'

In [ ]:
req_cols=["genres","movie_id","keywords","title","cast","crew","overview"]
movies=movies[req_cols]
movies

,genres,movie_id,keywords,title,cast,crew,overview
0,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de...","In the 22nd century, a paraplegic Marine is di..."
1,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de...","Captain Barbossa, long believed to be dead, ha..."
2,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de...",A cryptic message from Bond’s past sends him o...
3,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",The Dark Knight Rises,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de...",Following the death of District Attorney Harve...
4,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",John Carter,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de...","John Carter is a war-weary, former military ca..."
...,...,...,...,...,...,...,...
4804,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",9367,"[{""id"": 5616, ""name"": ""united states\u2013mexi...",El Mariachi,"[{""cast_id"": 1, ""character"": ""El Mariachi"", ""c...","[{""credit_id"": ""52fe44eec3a36847f80b280b"", ""de...",El Mariachi just wants to play his guitar and ...
4805,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 10749, ""...",72766,[],Newlyweds,"[{""cast_id"": 1, ""character"": ""Buzzy"", ""credit_...","[{""credit_id"": ""52fe487dc3a368484e0fb013"", ""de...",A newlywed couple's honeymoon is upended by th...
4806,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 18, ""nam...",231617,"[{""id"": 248, ""name"": ""date""}, {""id"": 699, ""nam...","Signed, Sealed, Delivered","[{""cast_id"": 8, ""character"": ""Oliver O\u2019To...","[{""credit_id"": ""52fe4df3c3a36847f8275ecf"", ""de...","""Signed, Sealed, Delivered"" introduces a dedic..."
4807,[],126186,[],Shanghai Calling,"[{""cast_id"": 3, ""character"": ""Sam"", ""credit_id...","[{""credit_id"": ""52fe4ad9c3a368484e16a36b"", ""de...",When ambitious New York attorney Sam is sent t...


In [ ]:
#extracting libraries after converting to string
i=0
'''
for label in range(len(ast.literal_eval((movies['keywords'][i])))):
  d=ast.literal_eval((movies['keywords'][0]))[label]['name']
  i+=1
  print(d)
'''


def func(obj):
  L=[]
  for i in ast.literal_eval(obj):
    L.append(i['name'])
  return L

def func1(obj):
  counter=0
  L=[]
  for i in ast.literal_eval(obj):
    if counter!=3:
      L.append(i['character'])
    else:
      break
    counter+=1
  return L


#movies['genres']=movies['genres'].apply(func)

movies.head()

print(len(movies))


4809


In [ ]:
movies['keywords']=movies['keywords'].apply(func)
#movies['genres']=movies['genres'].apply(func)
for x in range(5):
  print(ast.literal_eval(((movies['crew'][x])))[0])


{'credit_id': '52fe48009251416c750aca23', 'department': 'Editing', 'gender': 0, 'id': 1721, 'job': 'Editor', 'name': 'Stephen E. Rivkin'}
{'credit_id': '52fe4232c3a36847f800b579', 'department': 'Camera', 'gender': 2, 'id': 120, 'job': 'Director of Photography', 'name': 'Dariusz Wolski'}
{'credit_id': '54805967c3a36829b5002c41', 'department': 'Sound', 'gender': 2, 'id': 153, 'job': 'Original Music Composer', 'name': 'Thomas Newman'}
{'credit_id': '52fe4781c3a36847f81398c3', 'department': 'Sound', 'gender': 2, 'id': 947, 'job': 'Original Music Composer', 'name': 'Hans Zimmer'}
{'credit_id': '52fe479ac3a36847f813eaa3', 'department': 'Writing', 'gender': 2, 'id': 7, 'job': 'Screenplay', 'name': 'Andrew Stanton'}


In [ ]:
def fetch_director(text):
  L=[]
  for i in ast.literal_eval(text):
    if(i['job']=='Director'):
       L.append(i['name'])
       break
  return L











In [ ]:
movies['director']=movies['crew'].apply(fetch_director)




In [ ]:
movies.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 4809 entries, 0 to 4808
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   genres    4809 non-null   object
 1   movie_id  4809 non-null   int64 
 2   keywords  4809 non-null   object
 3   title     4809 non-null   object
 4   cast      4809 non-null   object
 5   crew      4809 non-null   object
 6   overview  4806 non-null   object
 7   director  4809 non-null   object
dtypes: int64(1), object(7)
memory usage: 467.2+ KB


In [ ]:
movies['cast']=movies['cast'].apply(func1)

In [ ]:
movies['genres']=movies['genres'].apply(func)


In [ ]:
#movies['crew']=movies['crew'].apply(func1)


In [ ]:
#movies['overview']=movies['overview'].apply([lambda x: x.split()])

In [ ]:
#movies['director']=movies['director'].apply(lambda x:[ i.replace(" ","") for i in x])
#i is used for iterating through the list
#list comprehension inside a lambda fucntion



movies['overview']=movies['overview'].apply(lambda x: x.split() if isinstance(x, str) else [])

movies.head()

,genres,movie_id,keywords,title,cast,crew,overview,director
0,"[Action, Adventure, Fantasy, Science Fiction]",19995,"[culture clash, future, space war, space colon...",Avatar,"[Jake Sully, Neytiri, Dr. Grace Augustine]","[{""credit_id"": ""52fe48009251416c750aca23"", ""de...","[In, the, 22nd, century,, a, paraplegic, Marin...",[James Cameron]
1,"[Adventure, Fantasy, Action]",285,"[ocean, drug abuse, exotic island, east india ...",Pirates of the Caribbean: At World's End,"[Captain Jack Sparrow, Will Turner, Elizabeth ...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de...","[Captain, Barbossa,, long, believed, to, be, d...",[Gore Verbinski]
2,"[Action, Adventure, Crime]",206647,"[spy, based on novel, secret agent, sequel, mi...",Spectre,"[James Bond, Blofeld, Madeleine]","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de...","[A, cryptic, message, from, Bond’s, past, send...",[Sam Mendes]
3,"[Action, Crime, Drama, Thriller]",49026,"[dc comics, crime fighter, terrorist, secret i...",The Dark Knight Rises,"[Bruce Wayne / Batman, Alfred Pennyworth, Jame...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de...","[Following, the, death, of, District, Attorney...",[Christopher Nolan]
4,"[Action, Adventure, Science Fiction]",49529,"[based on novel, mars, medallion, space travel...",John Carter,"[John Carter, Dejah Thoris, Sola]","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de...","[John, Carter, is, a, war-weary,, former, mili...",[Andrew Stanton]


In [ ]:
movies.drop(['crew'],axis=1)
#concatinating three pandas series into 1
#conoverting string to list using .split
movies['tag']=movies['overview']+movies['director']+movies['cast']+movies['keywords']+movies['genres']

In [ ]:
len(movies)

4809

In [ ]:

movies['tag'][0]

['In',
 'the',
 '22nd',
 'century,',
 'a',
 'paraplegic',
 'Marine',
 'is',
 'dispatched',
 'to',
 'the',
 'moon',
 'Pandora',
 'on',
 'a',
 'unique',
 'mission,',
 'but',
 'becomes',
 'torn',
 'between',
 'following',
 'orders',
 'and',
 'protecting',
 'an',
 'alien',
 'civilization.',
 'James Cameron',
 'Jake Sully',
 'Neytiri',
 'Dr. Grace Augustine',
 'culture clash',
 'future',
 'space war',
 'space colony',
 'society',
 'space travel',
 'futuristic',
 'romance',
 'space',
 'alien',
 'tribe',
 'alien planet',
 'cgi',
 'marine',
 'soldier',
 'battle',
 'love affair',
 'anti war',
 'power relations',
 'mind and soul',
 '3d',
 'Action',
 'Adventure',
 'Fantasy',
 'Science Fiction']

In [ ]:
movies['tag'][0]
#use join, but apply lambda function for all objects in series
#what is the signifiance of text and object parameters

movies['tag'] = movies['tag'].apply(lambda x: " ".join(x))


In [ ]:
movies['tag']

0       In the 22nd century, a paraplegic Marine is di...
1       Captain Barbossa, long believed to be dead, ha...
2       A cryptic message from Bond’s past sends him o...
3       Following the death of District Attorney Harve...
4       John Carter is a war-weary, former military ca...
                              ...                        
4804    El Mariachi just wants to play his guitar and ...
4805    A newlywed couple's honeymoon is upended by th...
4806    "Signed, Sealed, Delivered" introduces a dedic...
4807    When ambitious New York attorney Sam is sent t...
4808    Ever since the second grade when he first saw ...
Name: tag, Length: 4809, dtype: object

In [ ]:
#movies['overview']=movies['overview'].apply(lambda x: x.split())

In [ ]:
movies.tag=movies.tag.apply(lambda x: x.lower())

In [ ]:
#removing spaces and lowercasing both using lambda functions
movies.head()
req_colss=['title','movie_id','tag']
new_df=movies[req_colss]

In [ ]:
new_df['tag'][0]

'in the 22nd century, a paraplegic marine is dispatched to the moon pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization. james cameron jake sully neytiri dr. grace augustine culture clash future space war space colony society space travel futuristic romance space alien tribe alien planet cgi marine soldier battle love affair anti war power relations mind and soul 3d action adventure fantasy science fiction'

In [ ]:
#stemming(nlp)


new_df



,title,movie_id,tag
0,Avatar,19995,"in the 22nd century, a paraplegic marine is di..."
1,Pirates of the Caribbean: At World's End,285,"captain barbossa, long believed to be dead, ha..."
2,Spectre,206647,a cryptic message from bond’s past sends him o...
3,The Dark Knight Rises,49026,following the death of district attorney harve...
4,John Carter,49529,"john carter is a war-weary, former military ca..."
...,...,...,...
4804,El Mariachi,9367,el mariachi just wants to play his guitar and ...
4805,Newlyweds,72766,a newlywed couple's honeymoon is upended by th...
4806,"Signed, Sealed, Delivered",231617,"""signed, sealed, delivered"" introduces a dedic..."
4807,Shanghai Calling,126186,when ambitious new york attorney sam is sent t...


In [ ]:
from nltk.stem import PorterStemmer
ps=PorterStemmer()

#print((new_df['tag']))



def funcc(text):
  L1=[]
  for i in text.split():
    L1.append(ps.stem(i))
  return " ".join(L1)









In [ ]:
'''
i=0
for x in new_df['tag']:
  x=funcc(x)
  '''
new_df.loc[:, 'tag'] = new_df['tag'].apply(funcc)


#new_df['tag']=new_df['tag'].apply(funcc)
#you can only call literal eval on string

<ipython-input-201-ced3e073ef7d>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df.loc[:, 'tag'] = new_df['tag'].apply(funcc)


In [ ]:
funcc(d[0])

'in the 22nd century, a parapleg marin is dispatch to the moon pandora on a uniqu mission, but becom torn between follow order and protect an alien civilization.'

In [ ]:
d=movies['overview'].apply(lambda x:" ".join(x))
print(d[0])
print(new_df['tag'][0])
#yesssss
print(len(movies))

In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization.
in the 22nd century, a parapleg marin is dispatch to the moon pandora on a uniqu mission, but becom torn between follow order and protect an alien civilization. jame cameron jake sulli neytiri dr. grace augustin cultur clash futur space war space coloni societi space travel futurist romanc space alien tribe alien planet cgi marin soldier battl love affair anti war power relat mind and soul 3d action adventur fantasi scienc fiction
4809


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
#white-spaces and punctuation as token separators DURING TOKENIZATINO
#vectorization the general process of turning a collection of text documents into
#numerical feature vectors. This specific strategy (tokenization, counting and normalization)
 #is called the Bag of Words or “Bag of n-grams” representation
cv=CountVectorizer(max_features=50,stop_words='english')
vector=cv.fit_transform(new_df['tag']).toarray()

'''
below is a list of strings
corpus = [
...     'This is the first document.',
...     'This is the second second document.',
...     'And the third one.',
...     'Is this the first document?',
... ]

'''


#write in terms of ascending of frequency



"\nbelow is a list of strings\ncorpus = [\n...     'This is the first document.',\n...     'This is the second second document.',\n...     'And the third one.',\n...     'Is this the first document?',\n... ]\n\n"

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
similarity=cosine_similarity(vector)
similarity.shape
#cosine distance - how different two vectors are
#cosine similarity - how similar they are (1-cosine())



(4809, 4809)

In [ ]:
type(new_df['title'].values[0])


str

In [ ]:

#The normal weights and biases requires alot of calculaino

print(similarity)
def collapse(L):
    L1 = []
    for i in L:
        L1.append(i.replace(" ",""))
    return L1





[[1.         0.34815531 0.16012815 ... 0.21821789 0.         0.        ]
 [0.34815531 1.         0.1672484  ... 0.11396058 0.         0.10660036]
 [0.16012815 0.1672484  1.         ... 0.         0.         0.        ]
 ...
 [0.21821789 0.11396058 0.         ... 1.         0.15430335 0.13363062]
 [0.         0.         0.         ... 0.15430335 1.         0.57735027]
 [0.         0.10660036 0.         ... 0.13363062 0.57735027 1.        ]]


In [ ]:
movies['cast'] = movies['cast'].apply(collapse)
movies['crew'] = movies['crew'].apply(collapse)
movies['genres'] = movies['genres'].apply(collapse)
movies['keywords'] = movies['keywords'].apply(collapse)

In [ ]:
new_df[new_df['title'] == 'The Lego Movie'].index[0]

744

In [ ]:
col=new_df[new_df["title"]=='Bad Company']
print(col["tag"])

594    when a harvard-educ cia agent is kill dure an ...
Name: tag, dtype: object


In [ ]:

def recommend(movie):
    index = new_df[new_df['title'] == movie].index[0]
    distances = sorted(list(enumerate(similarity[index])),reverse=True,key = lambda x: x[1])
    for i in distances[1:6]:
        print(new_df.iloc[i[0]].title)


In [ ]:
new_df

,title,movie_id,tag
0,Avatar,19995,"in the 22nd century, a parapleg marin is dispa..."
1,Pirates of the Caribbean: At World's End,285,"captain barbossa, long believ to be dead, ha c..."
2,Spectre,206647,a cryptic messag from bond’ past send him on a...
3,The Dark Knight Rises,49026,follow the death of district attorney harvey d...
4,John Carter,49529,"john carter is a war-weary, former militari ca..."
...,...,...,...
4804,El Mariachi,9367,el mariachi just want to play hi guitar and ca...
4805,Newlyweds,72766,a newlyw couple' honeymoon is upend by the arr...
4806,"Signed, Sealed, Delivered",231617,"""signed, sealed, delivered"" introduc a dedic q..."
4807,Shanghai Calling,126186,when ambiti new york attorney sam is sent to s...


In [ ]:
recommend("The Dark Knight Rises")

Blood Ties
The Siege
Harper
Drive
Michael Clayton


In [ ]:
import re

# Compile a regular expression pattern
pattern = re.compile(r'\d+')  # This pattern matches one or more digits

# Use the compiled pattern for matching
result = pattern.findall('c3da2bc')  # Match the pattern against a string

print(result)

['3', '2']


In [ ]:
new

'newdata.csv'